In [2]:
from IPython.display import HTML
def colab_link():
    display(HTML(f'<a href="<your notebook link>" target="_parent"><img src="https://colab.research.google.com/drive/1SxJRqyE9XEPLfzf7fnLfywIjPdM0ah4n" alt="Open In Colab"/></a>'))
colab_link()

In [ ]:
# prompt: conicet to drive

from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


## Install YOLOv8

In [ ]:
!pip install ultralytics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.7/778.7 kB 10.8 MB/s eta 0:00:00


In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.23 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (AMD EPYC 7B12)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 29.9/225.8 GB disk)



## Install ByteTrack

In [ ]:
import os
HOME = os.getcwd()

In [ ]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


## Load pre-trained YOLOv8 model

In [ ]:
# settings
MODEL = "yolov8x.pt"

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

In [ ]:
!pip install ultralytics==8.0.196

##Install data from Roboflow :

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Y120uatJU1I0iFx7TWYu")
project = rf.workspace("ieeeroboticswsu").project("boxdetect-fj8wk")
version = project.version(1)
dataset = version.download("yolov8")



##yolo training:


In [ ]:
!yolo task=detect mode=train model=yolov8n data=data.yaml epochs=50 imgsz=224  amp=False

##save runs :

In [ ]:
output_dir = '/content/drive/My Drive/yolo_training_output_T5Bootcamp'

In [ ]:
!cp -r runs "$output_dir"

##validtion

In [ ]:
!yolo task=detect mode=val model=/content/drive/MyDrive/yolo_training_output_T5Bootcamp/detect/train2/weights/best.pt data=data.yaml epochs=50 imgsz=224  amp=False

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 268 layers, 68124531 parameters, 0 gradients, 257.4 GFLOPs
val: Scanning /content/boxdetect-1/valid/labels.cache... 198 images, 0 backgrounds, 0 corrupt: 100% 198/198 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 13/13 [04:19<00:00, 19.95s/it]
                   all        198        433      0.945       0.91      0.954      0.793
Speed: 0.9ms preprocess, 1286.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


## counter

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("/content/drive/MyDrive/yolo_training_output_T5Bootcamp/detect/train2/weights/best.pt")

# Video input path
video_path = '/content/WhatsApp Video 2024-05-23 at 2.09.03 PM.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Video output setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 'mp4v' codec for .mp4 files
output_path = '/content/output_video.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Calculate center line coordinates
center_x_line = frame_width // 2
line_coords = [(center_x_line, 0), (center_x_line, frame_height)]
line_color = (0, 0, 255)

# Initialize variables to keep track of box positions and IDs
box_ids = {}
next_box_id = 0
total_counted = 0
box_lifetime = 30  # Number of frames to keep a box in memory if not detected

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect objects in the frame
    detect_params = model.predict(source=[frame], conf=0.20, save=False, verbose=False)

    current_frame_boxes = []

    if detect_params and len(detect_params[0].boxes) > 0:
        for box in detect_params[0].boxes:
            bb = box.xyxy.cpu().numpy()[0]
            center_x = int((bb[0] + bb[2]) / 2)
            center_y = int((bb[1] + bb[3]) / 2)

            current_frame_boxes.append((center_x, center_y, bb))

            # Assign an ID to the box
            box_id = None
            for key, value in box_ids.items():
                if abs(value['coords'][0] - center_x) < 50 and abs(value['coords'][1] - center_y) < 50:
                    box_id = key
                    break

            if box_id is None:
                box_id = next_box_id
                next_box_id += 1
                box_ids[box_id] = {'coords': (center_x, center_y), 'counted': False, 'prev_state': 'left', 'lifetime': box_lifetime}

            box_ids[box_id]['coords'] = (center_x, center_y)
            box_ids[box_id]['lifetime'] = box_lifetime  # Reset lifetime

            # Count the box if it crosses the line
            if not box_ids[box_id]['counted']:
                if box_ids[box_id]['prev_state'] == 'left' and center_x >= center_x_line + 10:
                    total_counted += 1
                    box_ids[box_id]['counted'] = True
                    box_ids[box_id]['prev_state'] = 'right'
                elif center_x < center_x_line - 10:
                    box_ids[box_id]['prev_state'] = 'left'

    # Decrease the lifetime of all boxes and remove old boxes
    box_ids = {k: v for k, v in box_ids.items() if v['lifetime'] > 0}
    for box_id in box_ids:
        box_ids[box_id]['lifetime'] -= 1

    # Draw the bounding boxes and IDs
    for center_x, center_y, bb in current_frame_boxes:
        for box_id, value in box_ids.items():
            if value['coords'] == (center_x, center_y):
                cv2.rectangle(frame, (int(bb[0]), int(bb[1])), (int(bb[2]), int(bb[3])), (255, 0, 0), 2)
                cv2.putText(frame, f'ID: {box_id}', (int(bb[0]), int(bb[1]) - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
                # cv2.circle(frame, (center_x, center_y), 5, (0, 255, 0), -1)

    # Draw the center line
    # cv2.line(frame, line_coords[0], line_coords[1], line_color, 2)
    cv2.putText(frame, f'Total Counted: {total_counted}', (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()

print(f'Final Total Counted: {total_counted}')


Final Total Counted: 24
